#### 本章用到的数据集是泰坦尼克号的生存预测数据集，它包含了许多泰坦尼克号上的乘客的特征信息，以及该乘客最后是否生还。


| 列编号 | 特征名称       | 含义                     | 取值                   |
|--------|----------------|--------------------------|------------------------|
| 0      | 'PassengerId'  | 编号，从1开始            | 整数                   |
| 1      | 'Survived'     | 是否生还，1代表生还，0代表遇难 | 0、1                  |
| 2      | 'Pclass'       | 舱位等级                 | 0、1、2                |
| 3      | 'Name'         | 姓名                     | 字符串                 |
| 4      | 'Sex'          | 性别                     | 'male'、'female'       |
| 5      | 'Age'          | 年龄                     | 浮点数                 |
| 6      | 'SibSp'        | 登船的兄弟姐妹数量       | 整数                   |
| 7      | 'Parch'        | 登船的父母和子女数量     | 整数                   |
| 8      | 'Ticket'       | 船票编号                 | 字符串                 |
| 9      | 'Fare'         | 船票价格                 | 浮点数                 |
| 10     | 'Cabin'        | 所在船舱编号             | 字符串                 |
| 11     | 'Embarked'     | 登船港口，C代表瑟堡，S代表南安普敦，Q代表昆斯敦 | 'C'、'S'、'Q'         |


In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#读取数据
data=pd.read_csv('titanic/train.csv')

#查看数据信息和前五行具体内容，其中NaN代表数据缺失
print(data.info())
print(data[:5])

#删去编号
#columns=['PassengerId', 'Name', 'Ticket']: 指定要删除的列名。
#inplace=True: 表示直接在原 DataFrame 上修改，而不是返回一个新的 DataFrame。
data.drop(columns=['PassengerId','Name','Ticket'],inplace=True)

print(data[:5])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                      

在上面的算法介绍中，我们只考虑了特征取离散值的情况。在实践中，还有许多特征的取值是连续的，例如本数据集中的年龄和船票价格两项。对于这样的特征，我们可以根据数据的范围划出几个分类点 $x_1, \dots, x_K$，并按照取值与分类点的大小关系进行分类。具体来说，可以将数据分成以下 $K+1$ 类：

- $(-\infty, x_1]$
- $(x_1, x_2]$
- $\dots$
- $(x_K, +\infty)$

这样，我们就把连续的数据转化成了离散的取值。

In [39]:
feat_ranges = {} #字典
cont_feat = ['Age', 'Fare'] # 连续特征
bins = 10 # 分类点数

for feat in cont_feat:
    # 数据集中存在缺省值nan，需要用np.nanmin和np.nanmax,如果直接使用 min() 或 max()，当列中存在NaN时会抛出错误或返回 NaN
    # 计算当前特征列的最小值和最大值，忽略缺失值（NaN）。
    min_val = np.nanmin(data[feat])   
    max_val = np.nanmax(data[feat])   
    feat_ranges[feat] = np.linspace(min_val, max_val, bins).tolist()  #.tolist()将NumPy数组转换为Python列表，方便后续操作。
    print(feat, '：') # 查看分类点
    for spt in feat_ranges[feat]:
        print(f'{spt:.4f}')

Age ：
0.4200
9.2622
18.1044
26.9467
35.7889
44.6311
53.4733
62.3156
71.1578
80.0000
Fare ：
0.0000
56.9255
113.8509
170.7764
227.7019
284.6273
341.5528
398.4783
455.4037
512.3292


#### 介绍一下分类类型

分类类型是指数据的取值范围是有限的、离散的集合，而不是连续的数值。例如：

- 性别 ：`['male', 'female']`
- 颜色 ：`['red', 'green', 'blue']` 


分类类型的常见操作

 - 数据类型转换
   
   - 在 Pandas 中，可以将分类数据转换为 category 类型

    ``` python
    data['Sex'] = data['Sex'].astype('category')
    ```
 - 查看类别

   - 使用 `cat.categories` 查看所有类别：

   ``` python 
   print(data['Sex'].cat.categories) 
   #输出示例
   # Index(['female', 'male'], dtype='object')
   ```
 
 - 编码
   
   - 整数编码（Label Encoding）
     
     - 将类别按顺序映射为整数：
     ``` python
     data['Sex'] = data['Sex'].cat.codes
     #输出示例
     #female -> 0 male -> 1
     ```
   
   - 独热编码（One-Hot Encoding）

    - 使用 Pandas 的 get_dummies 方法：
     ``` python
     data = pd.get_dummies(data, columns=['Sex'])
     #输出示例
     #   Sex_female  Sex_male
     #0      0         1
     #1      1         0
     ```

In [40]:
# 只有有限取值的离散特征
cat_feat = ['Sex', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Embarked'] 

for feat in cat_feat:
    data[feat]=data[feat].astype('category') # 将特征列的数据类型转换为Pandas的category类型
    print(f'{feat}:{data[feat].cat.categories}') #查看类别
    data[feat]=data[feat].cat.codes.to_list() #将类别按顺序转化为整数
    ranges=list(set(data[feat]))  #使用 set(data[feat]) 获取当前特征的所有唯一取值。
    ranges.sort()             
    feat_ranges[feat]=ranges

Sex:Index(['female', 'male'], dtype='object')
Pclass:Index([1, 2, 3], dtype='int64')
SibSp:Index([0, 1, 2, 3, 4, 5, 8], dtype='int64')
Parch:Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
Cabin:Index(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31', 'A32',
       ...
       'E8', 'F E69', 'F G63', 'F G73', 'F2', 'F33', 'F38', 'F4', 'G6', 'T'],
      dtype='object', length=147)
Embarked:Index(['C', 'Q', 'S'], dtype='object')


In [41]:
#将所有缺省值替换为-1
data.fillna(-1,inplace=True)  
for feat in feat_ranges.keys():  # 遍历字典feat_ranges中的所有键（即特征名）。
    feat_ranges[feat]=[-1]+feat_ranges[feat] #使用列表拼接操作[-1] + feat_ranges[feat]，确保 -1 成为第一个元素。

#这行代码的作用是将 -1 添加到每个特征的取值范围中，确保缺失值的填充值也被视为合法的类别之一。

In [42]:
#划分数据集与测试集
np.random.seed(0)

feat_names=data.columns[1:]
label_name=data.columns[0]
print(feat_names)
print(label_name)

#重排下标
data=data.reindex(np.random.permutation(data.index))
ratio=0.8
split=int(ratio*len(data))

train_x=data[:split].drop(columns='Survived').to_numpy()
train_y=data['Survived'][:split].to_numpy()
test_x=data[split:].drop(columns='Survived').to_numpy()
test_y=data['Survived'][split:].to_numpy()

print('训练集大小:',len(train_x))
print('测试集大小:',len(test_x))
print('特征数大小:',train_x.shape[1])

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked'], dtype='object')
Survived
训练集大小: 712
测试集大小: 179
特征数大小: 8


In [43]:
class Node:
    def __init__(self):
        #内部结点的feat表示用来分类的特征编号，其数字与数据中的顺序对应
        #叶节点的feat表示该结点的分类结果
        self.feat=None
        #分类值列表，表示按照其中的值向子结点分类
        self.split=None
        #子节点列表，叶节点的child为空
        self.child=[]

In [ ]:
class DecisionTree:

    def __init__(self,X,Y,feat_ranges,lbd):
        self.root=Node()
        self.X=X
        self.Y=Y
        self.feat_ranges=feat_ranges #特征取值范围
        self.lbd=lbd  #正则化系数
        self.eps=1e-8 #防止log0的情况
        self.T=0  #记录叶节点数
        self.ID3(self.root,self.X,self.Y)

    #工具函数，计算 a*log a
    def aloga(self,a):
        return a*np.log2(a+self.eps)
    
    #计算某个子数据集的熵
    def entropy(self,Y):
        #统计每个类别出现的次数
        #返回的是一个包含两个元素的元组：
        # 1 唯一值  2 每个唯一值对应的出现次数。
        cnt=np.unique(Y,return_counts=True)[1]   #return_counts=True: 表示返回每个唯一值的出现次数。 
        N=len(Y)
        ent=-np.sum([self.aloga(Ni/N) for Ni in cnt])
        return ent
    #计算feat<=val划分数据集的信息增益

    def info_gain(self,X,Y,feat,val):
        #划分前的熵
        N=len(Y)
        if N==0 :
            return 0
        HX=self.entropy(Y)
        HXY=0 #H（X|Y）
        #分别计算H（X|X_F<=val）和分别计算H（X|X_F>val）
        Y_l=Y[X[:,feat]<=val]
        HXY+=len(Y_l)/len(Y)*self.entropy(Y_l)
        Y_r=Y[X[:,feat]>val]
        HXY+=len(Y_r)/len(Y)*self.entropy(Y_r)
        return HX-HXY
    
    #计算特征feat<=val本身的复杂度H_Y(X)
    def entropy_YX(self,X,Y,feat,val):
        HYX = 0
        N = len(Y)
        if N == 0:
            return 0
        Y_l = Y[X[:, feat] <= val]
        HYX += -self.aloga(len(Y_l) / N)
        Y_r = Y[X[:, feat] > val]
        HYX += -self.aloga(len(Y_r) / N)
        return HYX
    
    #计算用feat<=val划分数据集的信息增益率
    def info_gain_ratio(self,X,Y,feat,val):
        IG=self.info_gain(X,Y,feat,val)
        HYX=self.entropy_YX(X,Y,feat,val)
        return IG/HYX
        
    #用ID3算法递归分裂结点，构造决策树
    def ID3(self,node,X,Y):
        #判断是否已经分类完成
        if len(np.unique(Y))==1:
            node.feat=Y[0]
            self.T+=1
            return 
        
        #寻找最优分类特征和分类点
        best_IGR=0
        best_feat=None
        best_val=None
        for feat in range(len(feat_names)):
            for val in self.feat_ranges[feat_names[feat]]:
                IGR=self.info_gain_ratio(X,Y,feat,val)
                if IGR>best_IGR:
                    best_IGR=IGR
                    best_feat=feat
                    best_val=val
        #计算用best_feat<=best_val分类带来的代价函数变化
        #由于分裂叶结点只涉及该局部，我们只需要计算分裂前后该结点的代价函数
        #当前代价
        cur_cost=len(Y)*self.entropy(Y)+self.lbd
        #分裂后的代价，按best_feat的取值分裂统计
        #如果best_feat为None，说明最优的信息增益率为0
        #在分类也无法增加信息了，因此将new_cost设置为无穷大
        if best_feat is None:
            new_cost=np.inf
        else:
            new_cost=0
            X_feat=X[:,best_feat]
            #获取划分后的两部分，计算新的熵
            new_Y_l=Y[X_feat<=best_val]
            new_cost+=len(new_Y_l)*self.entropy(new_Y_l)
            new_Y_r=Y[X_feat>best_val]
            new_cost+=len(new_Y_r)*self.entropy(new_Y_r)
            #分裂后会有两个叶节点
            new_cost+=2*self.lbd
        
        if new_cost <= cur_cost:
            # 如果分裂后代价更小，那么执行分裂
            node.feat = best_feat
            node.split = best_val
            l_child = Node()
            l_X = X[X_feat <= best_val]
            l_Y = Y[X_feat <= best_val]
            self.ID3(l_child, l_X, l_Y)
            r_child = Node()
            r_X = X[X_feat > best_val]
            r_Y = Y[X_feat > best_val]
            self.ID3(r_child, r_X, r_Y)
            node.child = [l_child, r_child]
        else:
            # 否则将当前结点上最多的类别作为该结点的类别
            vals, cnt = np.unique(Y, return_counts=True)
            node.feat = vals[np.argmax(cnt)]
            self.T += 1
        
    def predict(self,x):
        node=self.root
        #从根结点开始向下寻找，找到叶结点结束
        while node.split is not None:
            if x[node.feat] <= node.split:
                node=node.child[0]
            else:
                node=node.child[1]
        
        #到达叶结点，返回类型
        return node.feat
    
    #计算在样本X，标签Y上的准确率
    def accuracy(self,X,Y):
        correct=0
        for x,y in zip(X,Y):
            pred=self.predict(x)
            if pred==y:
                correct+=1
        return correct/len(Y)        

In [45]:
DT = DecisionTree(train_x, train_y, feat_ranges, lbd=1.0)
print('叶结点数量：', DT.T)

# 计算在训练集和测试集上的准确率
print('训练集准确率：', DT.accuracy(train_x, train_y))
print('测试集准确率：', DT.accuracy(test_x, test_y))

叶结点数量： 23
训练集准确率： 0.8300561797752809
测试集准确率： 0.7262569832402235
